In [1]:
import _pickle as pickle
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import timeit

In [2]:
stats_keys = [
    "accuracy",
    "p_iwl",
    "diff last context",
    "ic_pred",
    "iw_pred",
]

In [3]:
variant_name = "simple_icl-fixed_g"

os.makedirs(
    "/home/chanb/src/simple_icl/cc_utils/agg_stats",
    exist_ok=True
)

results_dir = "/home/chanb/scratch/simple_icl/results/{}".format(variant_name)

In [ ]:
stats = []
for run_name in os.listdir(results_dir):
    tic = timeit.default_timer()

    prefix = "-".join(run_name.split("-")[:-9])
    seed = run_name.split("-")[-9]

    data = pickle.load(open(os.path.join(results_dir, run_name, "evaluation.pkl"), "rb"))
    for eval_name in data["stats"]:
        for stats_key in stats_keys:
            stats.append(
                dict(
                    variant=prefix,
                    seed=seed,
                    eval_name=eval_name,
                    stats_key=stats_key,
                    stats=data["stats"][eval_name][stats_key],
                )
            )

    toc = timeit.default_timer()
    print("Done {} in {}s".format(run_name, toc - tic))


Done high_prob_0.67-ground_truth_prob_1.0-seed_2-09-01-24_14_33_42-d83d532c-2362-401d-bec3-0fe7ebe9ea86 in 5.600146490964107s
Done high_prob_0.8-ground_truth_prob_0.1-seed_6-09-01-24_14_38_49-45505e22-e07c-4338-a719-e64bfdb7d3c2 in 5.159774504019879s
Done high_prob_0.9-ground_truth_prob_0.9-seed_8-09-01-24_14_39_59-9a2da965-d4bc-4f81-853b-2b7331bf4a73 in 4.991134651936591s
Done high_prob_0.8-ground_truth_prob_0.75-seed_3-09-01-24_14_36_04-98ce4c13-fabc-4d1e-b38e-964d526ec485 in 5.214175465982407s
Done high_prob_0.8-ground_truth_prob_0.1-seed_5-09-01-24_14_38_09-8360cb25-e560-45e3-9cf1-9a399085fd2a in 4.882570157991722s
Done high_prob_0.9-ground_truth_prob_0.25-seed_4-09-01-24_14_36_40-ab4112d1-873f-42bc-bb13-b3dd94cac5ac in 4.894950699992478s
Done high_prob_0.99-ground_truth_prob_0.0-seed_2-09-01-24_14_34_30-ac425b6e-62c3-469d-9980-66aaee14b2fe in 4.900218411930837s
Done high_prob_0.75-ground_truth_prob_0.1-seed_1-09-01-24_14_32_28-027c99b1-b5bc-40c5-a912-8dd3e02dce02 in 5.888120434014

In [ ]:
stats = pd.DataFrame(stats)

In [ ]:
pickle.dump(
    stats,
    open(os.path.join("/home/chanb/src/simple_icl/cc_utils/agg_stats", "{}.pkl".format(variant_name)), "wb")
)

In [ ]:
stats

In [ ]:
curr_stats = pd.DataFrame(stats["pretrain-sample_high_prob_class_only-start_pos_0"].tolist())

In [ ]:
curr_stats

In [ ]:
curr_stats["accuracy"]